# Расширенное тестирование модуля формирования портфеля

Этот ноутбук демонстрирует расширенный цикл работы с модулем tvr_service.pipeline.portfolio — от загрузки данных по инструментам до расчёта распределения капитала с использованием индикаторного инструмента и параметров.


## 1. Подготовка окружения

Импортируем необходимые библиотеки и подключим модуль портфеля. Добавляем каталог src в sys.path, чтобы можно было использовать код проекта без установки пакета.


In [1]:
import sys
from pathlib import Path

#========================
# import pandas as pd
# import numpy as np

# # Настройка отображения
# pd.set_option('display.max_rows', None)        # Показывать все строки
# pd.set_option('display.max_columns', None)     # Показывать все колонки  
# pd.set_option('display.width', None)           # Не ограничивать ширину
# pd.set_option('display.max_colwidth', 50)      # Ограничить ширину колонок для читаемости
# pd.set_option('display.expand_frame_repr', False)  # Не переносить DataFrame на новую строку
#========================

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from tvr_service.pipeline import (
    allocations_to_frame,
    build_portfolio,
    filter_by_suffix,
    filter_by_whitelist,
    load_securities,
    load_whitelist,
    )

import pandas as pd


## 2. Настройка параметров теста

Задаём базовые параметры: объём капитала, необходимость скачивания свежих данных с MOEX, путь к локальному CSV (если он есть), фильтрацию по суффиксу тикера, количество параметров и индикаторный инструмент.


In [2]:
CAPITAL = 4_00_000  # общий капитал, руб.
PREFER_REMOTE = True   # при True сначала пробуем скачать данные с MOEX
SEC_DATA_FILE = None   # можно указать путь к локальному sec_tvr.csv
SUFFIX_FILTER = 'Z5'   # например, 'H5', 'M5' и т.п. для фьючерсов
NUM_PARAMS = 1         # количество параметров для каждого инструмента в портфеле
SEC_IND = 'MMZ5'       # код индикаторного инструмента (пример)

# Параметры для расчета float значений
float_1_x = 0.01  # коэффициент для float_1 
float_2_x = 0.02   # коэффициент для float_2
float_3_x = 0.1   # значение для float_3
float_4_x = 0.   # значение для float_4
        # float_1 = max(1, int(sec_0_price / 100 * float_1_x))
        # float_2 = max(1, int(sec_0_price / 100 * float_2_x))
        # float_3 = float_3_x
        # float_4 = float_4_x

## 3. Загрузка справочника инструментов

Попробуем получить таблицу инструментов. Функция load_securities автоматически рассчитает поле full_price и добавит тикеры TI* (клонов TB*). В процессе будет использован список из интернета или локального файла.


In [3]:
securities = load_securities(sec_data_file=SEC_DATA_FILE, prefer_remote=PREFER_REMOTE)
print(f"Загружено инструментов: {len(securities)}")
# securities.head()


Загружено инструментов: 396


In [4]:
securities.head()

,SECID,MINSTEP,STEPPRICE,PREVSETTLEPRICE,INITIALMARGIN,BUYSELLFEE,SCALPERFEE,SHORTNAME,full_price,CODE,PRICE,SELLDEPO,base_code
0,AEH6,0.001,1.0,23.258,3752.24,1.08,0.54,AED-3.26,23258.0,AEH6,23.258,3752.24,AED
1,AEM6,0.001,1.0,24.551,3938.12,1.13,0.57,AED-6.26,24551.0,AEM6,24.551,3938.12,AED
2,AEZ5,0.001,1.0,22.587,3675.62,1.04,0.52,AED-12.25,22587.0,AEZ5,22.587,3675.62,AED
3,AFH6,1.000,1.0,5680.000,1489.37,1.13,0.57,AFLT-3.26,5680.0,AFH6,5680.000,1489.37,AFLT
4,AFZ5,1.000,1.0,5461.000,1410.00,1.08,0.54,AFLT-12.25,5461.0,AFZ5,5461.000,1410.00,AFLT


## 4. Получение цены индикаторного инструмента

Найдем цену индикаторного инструмента в справочнике и сохраним её в переменную SEC_IND_PRICE.


## 7.5. Обработка лучших параметров из best_params.csv

Загружаем и обрабатываем данные о лучших параметрах для каждого инструмента из файла best_params.csv, затем объединяем их с основным портфелем.


In [ ]:
# Загружаем данные из best_params.csv
best_params_path = PROJECT_ROOT / 'my_experiments' / 'best_params.csv'
best_params_df = pd.read_csv(best_params_path, sep=',')

print(f"Загружено записей из best_params.csv: {len(best_params_df)}")
print("Первые 5 записей:")
display(best_params_df.head())

print(f"\nУникальных инструментов в best_params: {best_params_df['Sec 0'].nunique()}")
print("Примеры инструментов:", best_params_df['Sec 0'].unique()[:10].tolist())


In [ ]:
# Обработка данных best_params.csv
print("=== ОБРАБОТКА BEST_PARAMS.CSV ===")

# 1. Добавляем 'Z5' к названию инструмента в колонке 'Sec 0'
best_params_df['Sec 0'] = best_params_df['Sec 0'] + 'Z5'

print(f"После добавления 'Z5' к инструментам:")
print("Примеры инструментов:", best_params_df['Sec 0'].unique()[:10].tolist())

# 2. Группируем по 'Sec 0' и выбираем максимальную Efficiency
best_params_grouped = best_params_df.groupby('Sec 0').agg({
    'PARAMS': lambda x: x.loc[best_params_df.loc[x.index, 'Efficiency'].idxmax()],
    'Efficiency': 'max'
}).reset_index()

print(f"\nПосле группировки и выбора максимальной Efficiency:")
print(f"Количество уникальных инструментов: {len(best_params_grouped)}")

# 3. Разлагаем колонку PARAMS на части
def parse_params(params_str):
    """Разбирает строку параметров и извлекает N и P"""
    try:
        # Разделяем по '_'
        parts = params_str.split('_')
        
        # Убираем первый элемент (0)
        if len(parts) > 1:
            # Берем второй элемент как N (1-й после удаления первого)
            N = float(parts[1]) if parts[1] != 'nan' else None
            
            # Берем третий элемент как P (2-й после удаления первого)  
            P = float(parts[2]) if len(parts) > 2 and parts[2] != 'nan' else None
            
            return N, P
        else:
            return None, None
    except (ValueError, IndexError):
        return None, None

# Применяем функцию к каждой строке
params_parsed = best_params_grouped['PARAMS'].apply(parse_params)
best_params_grouped['N'] = [x[0] for x in params_parsed]
best_params_grouped['P'] = [x[1] for x in params_parsed]

# 4. Оставляем только нужные колонки
best_params_final = best_params_grouped[['Sec 0', 'N', 'P']].copy()

print(f"\nПосле разбора параметров:")
print(f"Количество записей: {len(best_params_final)}")
print("Примеры обработанных данных:")
display(best_params_final.head(10))

# Проверяем, сколько значений N и P получилось
print(f"\nСтатистика по N и P:")
print(f"Непустых значений N: {best_params_final['N'].notna().sum()}")
print(f"Непустых значений P: {best_params_final['P'].notna().sum()}")
print(f"Пустых значений N: {best_params_final['N'].isna().sum()}")
print(f"Пустых значений P: {best_params_final['P'].isna().sum()}")


In [ ]:
# Объединение с expanded_portfolio_df
print("=== ОБЪЕДИНЕНИЕ С EXPANDED_PORTFOLIO_DF ===")

if 'expanded_portfolio_df' in locals() and not expanded_portfolio_df.empty:
    print(f"Размер expanded_portfolio_df: {len(expanded_portfolio_df)} строк")
    print(f"Уникальных инструментов в expanded_portfolio_df: {expanded_portfolio_df['Sec_0'].nunique()}")
    
    # Объединяем по Sec_0 (left join - оставляем все из expanded_portfolio_df)
    expanded_with_params = expanded_portfolio_df.merge(
        best_params_final, 
        left_on='Sec_0', 
        right_on='Sec 0', 
        how='left'
    )
    
    # Удаляем дублирующую колонку 'Sec 0'
    expanded_with_params = expanded_with_params.drop('Sec 0', axis=1)
    
    print(f"\nПосле объединения:")
    print(f"Размер результирующей таблицы: {len(expanded_with_params)} строк")
    print(f"Количество колонок: {len(expanded_with_params.columns)}")
    
    # Статистика по совпадениям
    matched_count = expanded_with_params['N'].notna().sum()
    unmatched_count = expanded_with_params['N'].isna().sum()
    
    print(f"\nСтатистика совпадений:")
    print(f"Совпавших инструментов: {matched_count}")
    print(f"Несовпавших инструментов: {unmatched_count}")
    
    # Показываем примеры совпавших и несовпавших
    print(f"\nПримеры совпавших инструментов:")
    matched_examples = expanded_with_params[expanded_with_params['N'].notna()][['Sec_0', 'N', 'P']].head()
    display(matched_examples)
    
    if unmatched_count > 0:
        print(f"\nПримеры несовпавших инструментов:")
        unmatched_examples = expanded_with_params[expanded_with_params['N'].isna()][['Sec_0']].head()
        display(unmatched_examples)
    
else:
    print("❌ Ошибка: expanded_portfolio_df не найден или пуст")
    print("Убедитесь, что выполнили все предыдущие шаги блокнота.")


In [ ]:
# Заполнение пустых значений N и P
print("=== ЗАПОЛНЕНИЕ ПУСТЫХ ЗНАЧЕНИЙ ===")

if 'expanded_with_params' in locals():
    # Заполняем пустые значения N и P значениями по умолчанию
    expanded_with_params['N'] = expanded_with_params['N'].fillna(0.001)
    expanded_with_params['P'] = expanded_with_params['P'].fillna(-0.00001)
    
    print("Заполнены пустые значения:")
    print("N = 0.001 (для всех пустых значений)")
    print("P = -0.00001 (для всех пустых значений)")
    
    # Проверяем результат
    print(f"\nПосле заполнения:")
    print(f"Пустых значений N: {expanded_with_params['N'].isna().sum()}")
    print(f"Пустых значений P: {expanded_with_params['P'].isna().sum()}")
    
    # Статистика по значениям N и P
    print(f"\nСтатистика по N:")
    print(f"Минимум: {expanded_with_params['N'].min():.6f}")
    print(f"Максимум: {expanded_with_params['N'].max():.6f}")
    print(f"Среднее: {expanded_with_params['N'].mean():.6f}")
    
    print(f"\nСтатистика по P:")
    print(f"Минимум: {expanded_with_params['P'].min():.6f}")
    print(f"Максимум: {expanded_with_params['P'].max():.6f}")
    print(f"Среднее: {expanded_with_params['P'].mean():.6f}")
    
    # Показываем финальный результат
    print(f"\nФинальная таблица:")
    print(f"Размер: {len(expanded_with_params)} строк, {len(expanded_with_params.columns)} колонок")
    print("Колонки:", list(expanded_with_params.columns))
    
    print(f"\nПримеры финальных данных:")
    display(expanded_with_params[['strategy_id', 'Sec_0', 'N', 'P', 'W_0', 'SM']].head(10))
    
    # Обновляем expanded_portfolio_df для дальнейшего использования
    expanded_portfolio_df = expanded_with_params.copy()
    print(f"\n✅ Обновлен expanded_portfolio_df с параметрами N и P")
    
else:
    print("❌ Ошибка: expanded_with_params не найден")
    print("Убедитесь, что предыдущие шаги выполнены успешно.")


In [5]:
# Поиск индикаторного инструмента в справочнике
indicator_security = securities[securities['SECID'] == SEC_IND]

if not indicator_security.empty:
    SEC_IND_PRICE = indicator_security['full_price'].iloc[0]
    SEC_IND_GO = indicator_security['INITIALMARGIN'].iloc[0]
    print(f"Индикаторный инструмент {SEC_IND} найден. Цена: {SEC_IND_PRICE:,.2f} руб.")
    print(f"Индикаторный инструмент {SEC_IND} найден. ГО: {SEC_IND_GO:,.2f} руб.")
    print(f"Краткое название: {indicator_security['SHORTNAME'].iloc[0]}")
else:
    print(f"Индикаторный инструмент {SEC_IND} не найден в справочнике!")
    print("Доступные инструменты с похожими кодами:")
    similar_securities = securities[securities['SECID'].str.contains(SEC_IND[:3], case=False)]
    if not similar_securities.empty:
        print(similar_securities[['SECID', 'SHORTNAME']].head())
    else:
        print("Похожих инструментов не найдено.")
    SEC_IND_PRICE = None


Индикаторный инструмент MMZ5 найден. Цена: 27,169.00 руб.
Индикаторный инструмент MMZ5 найден. ГО: 3,225.95 руб.
Краткое название: MXI-12.25


## 5. Фильтрация справочника

Сначала ограничиваем список инструментов по выбранному `SUFFIX_FILTER`, затем применяем whitelist. Так можно сузить вселенную до нужной серии и далее оставить только интересующие базовые активы.


In [6]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

# Фильтрация по суффиксу
base_selection = filter_by_suffix(securities, SUFFIX_FILTER)
print(f'Инструментов после фильтра по суффиксу {SUFFIX_FILTER}: {len(base_selection)}')

# Фильтрация по whitelist
if whitelist and not base_selection.empty:
    filtered_securities = filter_by_whitelist(base_selection, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_securities)}')
elif whitelist:
    filtered_securities = pd.DataFrame(columns=base_selection.columns if not base_selection.empty else securities.columns)
    print('После фильтра по суффиксу подходящих инструментов нет.')
else:
    filtered_securities = base_selection

if not filtered_securities.empty:
    print('Примеры отфильтрованных инструментов:')
    display(filtered_securities[['SECID', 'SHORTNAME', 'base_code', 'full_price']].head())
    
    # Проверка уникальности по base_code
    print(f"\nПроверка уникальности по base_code:")
    unique_base_codes = filtered_securities['base_code'].nunique()
    total_instruments = len(filtered_securities)
    print(f"Уникальных base_code: {unique_base_codes}")
    print(f"Общее количество инструментов: {total_instruments}")
    
    if unique_base_codes != total_instruments:
        print("⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!")
        duplicates = filtered_securities.groupby('base_code').size()
        duplicates = duplicates[duplicates > 1]
        print(f"Дублирующиеся base_code ({len(duplicates)}):")
        for base_code, count in duplicates.head(10).items():
            print(f"  {base_code}: {count} инструментов")
            # Показываем примеры дубликатов
            examples = filtered_securities[filtered_securities['base_code'] == base_code][['SECID', 'SHORTNAME', 'base_code']]
            print(f"    Примеры: {examples['SECID'].tolist()}")
    else:
        print("✅ Все base_code уникальны")
        
else:
    print('После фильтрации инструментов не найдено.')


Инструментов в whitelist: 43
Инструментов после фильтра по суффиксу Z5: 132
Инструментов после применения whitelist: 44
Примеры отфильтрованных инструментов:


,SECID,SHORTNAME,base_code,full_price
0,AFZ5,AFLT-12.25,AFLT,5461.0
1,AKZ5,AFKS-12.25,AFKS,13602.0
2,ALZ5,ALRS-12.25,ALRS,4313.0
3,ASZ5,ASTR-12.25,ASTR,304.0
4,BSZ5,BSPB-12.25,BSPB,3365.0



Проверка уникальности по base_code:
Уникальных base_code: 43
Общее количество инструментов: 44
⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!
Дублирующиеся base_code (1):
  T: 2 инструментов
    Примеры: ['TBZ5', 'TIZ5']


## 5.2. Альтернативный диалог (без ipywidgets)

Если ipywidgets недоступны, используем простой текстовый интерфейс для выбора дубликатов.

In [7]:
def remove_duplicates_simple(df, base_code_column='base_code'):
    """
    Простая функция для удаления дублирующихся инструментов по base_code.
    Использует текстовый интерфейс вместо ipywidgets.
    
    Args:
        df: DataFrame с инструментами
        base_code_column: название колонки для проверки дубликатов
    
    Returns:
        DataFrame без дубликатов
    """
    import pandas as pd
    
    # Находим дубликаты
    duplicates = df.groupby(base_code_column).size()
    duplicates = duplicates[duplicates > 1]
    
    if len(duplicates) == 0:
        print("✅ Дубликатов не найдено!")
        return df
    
    print(f"🔍 Найдено {len(duplicates)} групп дубликатов:")
    
    indices_to_remove = []
    
    for base_code in duplicates.index:
        group_data = df[df[base_code_column] == base_code]
        print(f"\n📋 Группа '{base_code}' ({len(group_data)} инструментов):")
        
        # Показываем инструменты в группе
        for i, (idx, row) in enumerate(group_data.iterrows()):
            print(f"  {i+1}. {row['SECID']} - {row['SHORTNAME']} (цена: {row['full_price']})")
        
        # Запрашиваем выбор
        print(f"\n❓ Какие инструменты из группы '{base_code}' удалить?")
        print("   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:")
        
        try:
            choice = input(2).strip()
            
            if choice.lower() == 'all':
                # Удаляем все кроме первого
                indices_to_remove.extend(group_data.index[1:].tolist())
                print(f"   ✅ Будет удалено {len(group_data)-1} инструментов (останется первый)")
            elif choice:
                # Парсим номера
                numbers = [int(x.strip()) for x in choice.split(',') if x.strip().isdigit()]
                if numbers:
                    # Проверяем валидность номеров
                    valid_numbers = [n for n in numbers if 1 <= n <= len(group_data)]
                    if valid_numbers:
                        # Получаем индексы для удаления
                        indices_to_remove.extend([group_data.index[n-1] for n in valid_numbers])
                        print(f"   ✅ Будет удалено {len(valid_numbers)} инструментов")
                    else:
                        print("   ⚠️ Некорректные номера, пропускаем группу")
                else:
                    print("   ⚠️ Некорректный ввод, пропускаем группу")
            else:
                print("   ⏭️ Группа пропущена")
                
        except (ValueError, KeyboardInterrupt):
            print("   ⚠️ Некорректный ввод или прерывание, пропускаем группу")
            continue
    
    if indices_to_remove:
        print(f"\n🗑️ Удаляем {len(indices_to_remove)} выбранных инструментов...")
        
        # Удаляем выбранные инструменты
        df_cleaned = df.drop(indices_to_remove).reset_index(drop=True)
        
        print(f"✅ Очистка завершена!")
        print(f"   Исходное количество: {len(df)}")
        print(f"   После очистки: {len(df_cleaned)}")
        print(f"   Удалено: {len(df) - len(df_cleaned)}")
        
        # Проверяем результат
        remaining_duplicates = df_cleaned.groupby(base_code_column).size()
        remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]
        
        if len(remaining_duplicates) == 0:
            print("✅ Все дубликаты успешно удалены!")
        else:
            print(f"⚠️ Остались дубликаты: {len(remaining_duplicates)} групп")
        
        return df_cleaned
    else:
        print("ℹ️ Ничего не выбрано для удаления.")
        return df

# Попробуем использовать простую версию, если интерактивная не работает
try:
    if 'filtered_securities' in locals() and not filtered_securities.empty:
        print("🔧 Запускаем простой диалог удаления дубликатов...")
        filtered_securities_cleaned = remove_duplicates_simple(filtered_securities)
        
        # Обновляем переменную
        filtered_securities = filtered_securities_cleaned
        print(f"\n📊 Итоговое количество инструментов: {len(filtered_securities)}")
    else:
        print("❌ Нет данных для обработки дубликатов")
except Exception as e:
    print(f"❌ Ошибка при обработке дубликатов: {e}")
    print("Продолжаем с исходными данными...")


🔧 Запускаем простой диалог удаления дубликатов...
🔍 Найдено 1 групп дубликатов:

📋 Группа 'T' (2 инструментов):
  1. TBZ5 - T-12.25 (цена: 3094.0)
  2. TIZ5 - T-12.25 (цена: 3094.0)

❓ Какие инструменты из группы 'T' удалить?
   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:
   ✅ Будет удалено 1 инструментов

🗑️ Удаляем 1 выбранных инструментов...
✅ Очистка завершена!
   Исходное количество: 44
   После очистки: 43
   Удалено: 1
✅ Все дубликаты успешно удалены!

📊 Итоговое количество инструментов: 43


## 6. Создание основы для построения портфеля

Создаем основу для портфеля с расчетом весовых коэффициентов относительно индикаторного инструмента.


In [8]:
if SEC_IND_PRICE is not None and not filtered_securities.empty:
    print(f"Создаем основу портфеля с {NUM_PARAMS} параметрами для каждого инструмента")
    print(f"Индикаторный инструмент: {SEC_IND} (цена: {SEC_IND_PRICE:,.2f} руб.)")
    
    # Создаем список для хранения данных портфеля
    portfolio_basis = []
    
    for idx, row in filtered_securities.iterrows():
        sec_0 = row['SECID']
        sec_0_price = row['full_price']
        sec_0_GO = row['INITIALMARGIN']
        sec_1 = SEC_IND
        sec_1_GO = SEC_IND_GO
        
        # Расчет весового коэффициента W_0
        W_0 = (sec_0_price / SEC_IND_PRICE * 1000000) // 1000 / 1000
        
        # Расчет float значений
        float_1 = max(1, int(sec_0_price / 100 * float_1_x))
        float_2 = max(1, int(sec_0_price / 100 * float_2_x))
        float_3 = float_3_x
        float_4 = float_4_x

        float_1_sh = float_1*4
        float_2_sh = max(1, int(sec_0_price / 100 * 0.13))
        float_3_sh = 0.2
        float_4_sh = 0.8
        
        # Формирование колонки SM
        SM = f"{float_1};{float_2};{float_3};{float_4}"
        SM_short = f"{float_1_sh};{float_2_sh};{float_3_sh};{float_4_sh}"
        
        portfolio_basis.append({
            'Sec_0': sec_0,
            'sec_0_price': sec_0_price,
            'sec_0_GO': sec_0_GO,
            'sec_1': sec_1,
            'W_0': W_0,
            'SM': SM,
            'SM_sh': SM_short,
            'sec_1_GO':sec_1_GO
        })
    
    # Преобразуем в DataFrame
    portfolio_basis_df = pd.DataFrame(portfolio_basis)
    
    print(f"\nСоздана основа портфеля для {len(portfolio_basis_df)} инструментов")
    print("\nПримеры записей:")
    # display(portfolio_basis_df.head())
    
    # Статистика по весовым коэффициентам
    print(f"\nСтатистика весовых коэффициентов W_0:")
    print(f"Минимум: {portfolio_basis_df['W_0'].min():.6f}")
    print(f"Максимум: {portfolio_basis_df['W_0'].max():.6f}")
    print(f"Среднее: {portfolio_basis_df['W_0'].mean():.6f}")
    print(f"Медиана: {portfolio_basis_df['W_0'].median():.6f}")
    
else:
    print("Ошибка: Не удалось создать основу портфеля.")
    if SEC_IND_PRICE is None:
        print("- Индикаторный инструмент не найден")
    if filtered_securities.empty:
        print("- Нет отфильтрованных инструментов")


Создаем основу портфеля с 1 параметрами для каждого инструмента
Индикаторный инструмент: MMZ5 (цена: 27,169.00 руб.)

Создана основа портфеля для 43 инструментов

Примеры записей:

Статистика весовых коэффициентов W_0:
Минимум: 0.011000
Максимум: 4.149000
Среднее: 0.625930
Медиана: 0.201000


## 7. Размножение строк по NUM_PARAMS

Создаем strategy_id для каждой комбинации инструмента и параметра, размножая строки по количеству параметров.


In [9]:
if 'portfolio_basis_df' in locals() and not portfolio_basis_df.empty:
    print("Размножаем строки по NUM_PARAMS...")
    
    # Создаем список параметров от 1 до NUM_PARAMS
    params_list = list(range(1, NUM_PARAMS + 1))
    print(f"Список параметров: {params_list}")
    
    # Создаем размноженные строки
    expanded_portfolio = []
    
    for idx, row in portfolio_basis_df.iterrows():
        for param_num in params_list:
            # Создаем strategy_id
            strategy_id = f"{row['Sec_0']}_{param_num}"
            
            # Копируем все данные строки
            expanded_row = row.copy()
            expanded_row['strategy_id'] = strategy_id
            expanded_row['param_num'] = param_num
            
            # Переупорядочиваем колонки, чтобы strategy_id была первой
            expanded_row = expanded_row.reindex(['strategy_id'] + [col for col in expanded_row.index if col != 'strategy_id'])
            
            expanded_portfolio.append(expanded_row)
    
    # Преобразуем в DataFrame
    expanded_portfolio_df = pd.DataFrame(expanded_portfolio)
    
    print(f"\nРазмножение завершено:")
    print(f"Исходных инструментов: {len(portfolio_basis_df)}")
    print(f"Параметров на инструмент: {NUM_PARAMS}")
    print(f"Итого строк: {len(expanded_portfolio_df)}")
    print(f"Ожидалось: {len(portfolio_basis_df) * NUM_PARAMS}")
    
    print("\nПримеры strategy_id:")
    sample_strategies = expanded_portfolio_df['strategy_id'].head(10).tolist()
    print(sample_strategies)
    
    print("\nПримеры записей:")
    display(expanded_portfolio_df[['strategy_id', 'Sec_0', 'param_num', 'W_0', 'SM']].head())
    
    # Проверяем уникальность strategy_id
    unique_strategies = expanded_portfolio_df['strategy_id'].nunique()
    total_strategies = len(expanded_portfolio_df)
    print(f"\nПроверка уникальности:")
    print(f"Уникальных strategy_id: {unique_strategies}")
    print(f"Общее количество строк: {total_strategies}")
    print(f"Совпадают: {unique_strategies == total_strategies}")
    
else:
    print("Ошибка: Не удалось размножить строки.")
    print("Убедитесь, что основа портфеля была создана успешно.")

expanded_portfolio_df = expanded_portfolio_df.reset_index(drop=True)


Размножаем строки по NUM_PARAMS...
Список параметров: [1]

Размножение завершено:
Исходных инструментов: 43
Параметров на инструмент: 1
Итого строк: 43
Ожидалось: 43

Примеры strategy_id:
['AFZ5_1', 'AKZ5_1', 'ALZ5_1', 'ASZ5_1', 'BSZ5_1', 'CHZ5_1', 'FLZ5_1', 'GKZ5_1', 'GZZ5_1', 'HDZ5_1']

Примеры записей:


,strategy_id,Sec_0,param_num,W_0,SM
0,AFZ5_1,AFZ5,1,0.201,1;1;0.1;0.0
1,AKZ5_1,AKZ5,1,0.500,1;2;0.1;0.0
2,ALZ5_1,ALZ5,1,0.158,1;1;0.1;0.0
3,ASZ5_1,ASZ5,1,0.011,1;1;0.1;0.0
4,BSZ5_1,BSZ5,1,0.123,1;1;0.1;0.0



Проверка уникальности:
Уникальных strategy_id: 43
Общее количество строк: 43
Совпадают: True


In [10]:
expanded_portfolio_df.head()

,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,SM_sh,sec_1_GO,param_num
0,AFZ5_1,AFZ5,5461.0,1410.00,MMZ5,0.201,1;1;0.1;0.0,4;7;0.2;0.8,3225.95,1
1,AKZ5_1,AKZ5,13602.0,3548.96,MMZ5,0.500,1;2;0.1;0.0,4;17;0.2;0.8,3225.95,1
2,ALZ5_1,ALZ5,4313.0,1137.45,MMZ5,0.158,1;1;0.1;0.0,4;5;0.2;0.8,3225.95,1
3,ASZ5_1,ASZ5,304.0,103.39,MMZ5,0.011,1;1;0.1;0.0,4;1;0.2;0.8,3225.95,1
4,BSZ5_1,BSZ5,3365.0,1204.00,MMZ5,0.123,1;1;0.1;0.0,4;4;0.2;0.8,3225.95,1


## 8. Формирование финального каркаса портфеля

Создаем финальную структуру портфеля с анализом результатов.


## 8.1. Тестирование новой функции build_portfolio_advanced

Теперь протестируем новую функцию `build_portfolio_advanced`, которая может работать с любым DataFrame и правильно распределять капитал по группам инструментов и параметрам.


In [ ]:

# Импортируем новую функцию
from tvr_service.pipeline import build_portfolio_advanced

if 'expanded_portfolio_df' in locals() and not expanded_portfolio_df.empty:
    print("⚠️  expanded_portfolio_df не найден.")
    

    
    # Теперь применяем новую функцию
    portfolio_with_allocation = build_portfolio_advanced(
        data=expanded_portfolio_df,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_GO",
        param_num_column="param_num",
        ensure_min_contract=True
    )
    
    print(f"Результат: {len(portfolio_with_allocation)} строк")
    print("\nПример результата:")
    display(portfolio_with_allocation[['strategy_id', 'Sec_0', 'param_num', 'sec_0_price', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].head(10))
    
else:
    print("❌ Ошибка: Не найдены необходимые данные.")
    print("Доступные переменные:")
    available_vars = [var for var in locals() if 'portfolio' in var.lower() or 'expanded' in var.lower()]
    if available_vars:
        print(f"  - {', '.join(available_vars)}")
    else:
        print("  - Нет переменных с 'portfolio' или 'expanded' в названии")
    print("\nУбедитесь, что выполнили все предыдущие шаги блокнота.")

# Анализ результатов распределения капитала
if 'portfolio_with_allocation' in locals():
    print("=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ КАПИТАЛА ===\n")
    
    # Общая статистика
    total_used = portfolio_with_allocation['used_capital'].sum()
    total_unused = portfolio_with_allocation['unused_capital'].sum()
    utilization_rate = total_used / CAPITAL * 100
    
    print(f"Общий капитал: {CAPITAL:,.0f} руб.")
    print(f"Использовано: {total_used:,.0f} руб. ({utilization_rate:.1f}%)")
    print(f"Неиспользовано: {total_unused:,.0f} руб. ({100-utilization_rate:.1f}%)")
    
    # Статистика по инструментам
    print(f"\n=== СТАТИСТИКА ПО ИНСТРУМЕНТАМ ===")
    instrument_stats = portfolio_with_allocation.groupby('Sec_0').agg({
        'allocation': 'first',  # allocation одинаковая для всех строк инструмента
        'estimated_lots': 'sum',
        'used_capital': 'sum',
        'unused_capital': 'sum',
        'param_num': 'count'
    }).rename(columns={'param_num': 'param_count'})
    
    print(f"Уникальных инструментов: {len(instrument_stats)}")
    print(f"Капитал на инструмент: {CAPITAL / len(instrument_stats):,.0f} руб.")
    print("\nТоп-5 инструментов по используемому капиталу:")
    top_instruments = instrument_stats.nlargest(44, 'used_capital')[['used_capital', 'estimated_lots', 'param_count']]
    display(top_instruments)
    
    
    
else:
    print("Переменная portfolio_with_allocation не найдена.")


In [ ]:
portfolio_with_allocation


In [ ]:
## 9. Формирование финальной таблицы портфеля

# Создаем финальную таблицу со всеми исходными колонками и добавленным столбцом V_0
if 'portfolio_with_allocation' in locals():
    print("Создаем финальную таблицу портфеля...")
    
    # Копируем все данные из исходной таблицы
    Final_portfolio_all = expanded_portfolio_df.copy()
    
    # Добавляем столбец V_0 с данными из estimated_lots
    Final_portfolio_all['V_0'] = portfolio_with_allocation['estimated_lots']
    
       
else:
    print("❌ Ошибка: portfolio_with_allocation не найден.")
    print("Убедитесь, что предыдущие ячейки выполнены успешно.")


In [ ]:
Final_portfolio_all.to_csv('porfolio_base.csv', sep = ',')

In [ ]:
from pandas import read_csv


port_base = read_csv('porfolio_base.csv', header=0)

In [ ]:
port_base